In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 3090


In [2]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [3]:
import json
with open('train.json', 'r') as f:
    data = json.load(f)

question_dic = {}
sparql_dic = {}

for x in range(4000):
    question_dic[x] = data[x]['corrected_question']
    sparql_dic[x] = data[x]['sparql_query']

In [4]:
with open('test.json', 'r') as f:
    data = json.load(f)

test_question_dic = {}
test_sparql_dic = {}

for x in range(len(data)):
    test_question_dic[x] = data[x]['corrected_question']
    test_sparql_dic[x] = data[x]['sparql_query']

In [5]:
def searchPredicatesWithoutType2(query):
    result = []

    li = query.split()
    index = 0
    while index < len(li):
        li[index] = li[index].strip()
        index += 1

    while '.' in li:
        dot_index = li.index('.')
        li.pop(dot_index)

    where_index = 0
    if 'where' in li:
        where_index = li.index('where')
    else:
        where_index = li.index('WHERE')

    if li[where_index + 1] == '{':
        next_predicate = where_index + 3
    else:
        next_predicate = where_index + 2

    while next_predicate < len(li):
        if 'http://dbpedia.org/ontology/' in li[next_predicate]:
            result.append(li[next_predicate][29:-1])
        elif 'http://dbpedia.org/property/' in li[next_predicate]:
            result.append(li[next_predicate][29:-1])

        next_predicate += 3

    return result

In [6]:
#build training and testing samples
def buildSamples(q_dic, s_dic):
    nlq = []
    predicates = []
    for x in range(len(q_dic)):
        current_predicates = searchPredicatesWithoutType2(s_dic[x])
        nlq.append(q_dic[x])
        predicates.append(current_predicates)
    return (nlq, predicates)

train_nlq, train_p = buildSamples(question_dic, sparql_dic)
test_nlq, test_p = buildSamples(test_question_dic, test_sparql_dic)

In [7]:
#check question and its correspoding predicates
q_id = 4
print(train_nlq[q_id])
print(train_p[q_id])

How many races have the horses bred by Jacques Van't Hart participated in?
['breeder', 'race']


In [8]:
# build a predicates-index dictionary
def build_labels_dic(p_list1, p_list2):
    res_dic = {}
    count = 0
    for x in p_list1:
        for y in x:
            if y not in res_dic:
                res_dic[y] = count
                count += 1
            
    for x in p_list2:
        for y in x:
            if y not in res_dic:
                res_dic[y] = count
                count += 1
            
    return res_dic

label_dic = build_labels_dic(train_p, test_p)

In [9]:
#append one-hot labels to a labels list
def label_to_index_list(l_list, l_dic):
    res_list = []
    for x in l_list:
        current_one_hot = [0] * len(l_dic)
        for y in x:
            current_one_hot[l_dic[y]] = 1
        res_list.append(current_one_hot)
    return res_list

In [10]:
train_labels = label_to_index_list(train_p, label_dic)
test_labels = label_to_index_list(test_p, label_dic)

# Tokenize Train/Test samples

In [11]:
train_input_ids = []
train_attention_masks = []

for sent in train_nlq:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_nlq[0])
print('Token IDs:', train_input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  How many movies did Stanley Kubrick direct?
Token IDs: tensor([  101,  2129,  2116,  5691,  2106,  6156, 13970, 25646,  3622,  1029,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [12]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
test_input_ids = []
test_attention_masks = []

# For every sentence...
for sent in test_nlq:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.tensor(test_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', test_nlq[0])
print('Token IDs:', test_input_ids[0])

Original:  Which architect of Marine Corps Air Station Kaneohe Bay was also tenant of New Sanno hotel /'
Token IDs: tensor([  101,  2029,  4944,  1997,  3884,  3650,  2250,  2276,  8472, 11631,
         2063,  3016,  2001,  2036, 16713,  1997,  2047,  2624,  3630,  3309,
         1013,  1005,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [13]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
# Calculate the number of samples to include in each set.
train_size = len(dataset)
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

4,000 training samples
    0 validation samples


In [14]:
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

test_size = len(test_dataset)
val_size = len(test_dataset) - test_size

test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

print('{:>5,} testing samples'.format(test_size))
print('{:>5,} validation samples'.format(val_size))

1,000 testing samples
    0 validation samples


In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

In [16]:
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size
        )

In [17]:
from transformers import BertPreTrainedModel, AdamW, BertConfig, BertModel
import torch.nn as nn
class BertForSequenceClassification2(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            elif self.num_labels == 437:
                loss_fct = nn.BCEWithLogitsLoss()
                labels = labels.float()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        '''
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
        '''
        return (loss, logits)

In [18]:
model = BertForSequenceClassification2.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 437, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification2: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification2 from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification2 from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification2 were not initialized from the model checkpoin

BertForSequenceClassification2(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [19]:
# fix first 9 layers of BERT
to_fix = ['0','1','2','3', '4', '5', '6', '7', '8']
fix_list = []
for x in to_fix:
    fix_list.append('encoder.layer.' + x + '.')
print(fix_list)
for k, x in model.named_parameters():
    for y in fix_list:
        if y in k:
            x.requres_grad = False

['encoder.layer.0.', 'encoder.layer.1.', 'encoder.layer.2.', 'encoder.layer.3.', 'encoder.layer.4.', 'encoder.layer.5.', 'encoder.layer.6.', 'encoder.layer.7.', 'encoder.layer.8.']


In [20]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [21]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [22]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [23]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
import random
import numpy as np

training_stats = []

total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        logits = outputs[1]
        loss = outputs[0]

        total_train_loss += loss.item()
        
        

        # Perform a backward pass to calculate the gradients.
        loss.backward()
        #print('current loss : ' + str(loss.item()))

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


======== Epoch 1 / 2 ========
Training...
  Batch    40  of    250.    Elapsed: 0:00:11.
  Batch    80  of    250.    Elapsed: 0:00:20.
  Batch   120  of    250.    Elapsed: 0:00:30.
  Batch   160  of    250.    Elapsed: 0:00:40.
  Batch   200  of    250.    Elapsed: 0:00:49.
  Batch   240  of    250.    Elapsed: 0:00:59.

  Average training loss: 0.29
  Training epcoh took: 0:01:01

======== Epoch 2 / 2 ========
Training...
  Batch    40  of    250.    Elapsed: 0:00:10.
  Batch    80  of    250.    Elapsed: 0:00:18.
  Batch   120  of    250.    Elapsed: 0:00:28.
  Batch   160  of    250.    Elapsed: 0:00:37.
  Batch   200  of    250.    Elapsed: 0:00:46.
  Batch   240  of    250.    Elapsed: 0:00:56.

  Average training loss: 0.09
  Training epcoh took: 0:00:58


In [25]:
from sklearn import metrics
def flat_accuracy(logits, labels):
    th = 0.1
    target = labels.cpu().detach().numpy().tolist()
    output = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist()) >= th
    return metrics.accuracy_score(target, output)

In [27]:
model.eval()

total_eval_accuracy = 0

for step, batch in enumerate(test_dataloader):
    if step == 1:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():        


            seq_result = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                              labels = b_labels)

            logits = seq_result[1]
            loss = seq_result[0]

            print(logits[0].sigmoid())

            total_eval_accuracy += flat_accuracy(logits, b_labels)
        
print(total_eval_accuracy / len(test_dataloader))

tensor([0.0820, 0.0627, 0.0605, 0.0985, 0.0861, 0.1139, 0.0596, 0.0463, 0.0458,
        0.0667, 0.0910, 0.0737, 0.0725, 0.0259, 0.0408, 0.0562, 0.0471, 0.0638,
        0.0635, 0.0723, 0.0415, 0.1273, 0.1004, 0.0578, 0.0700, 0.0479, 0.1035,
        0.0916, 0.0737, 0.0984, 0.0634, 0.0560, 0.0511, 0.0488, 0.0512, 0.0668,
        0.0578, 0.0763, 0.0813, 0.0810, 0.1238, 0.0624, 0.0493, 0.0490, 0.0577,
        0.0533, 0.0574, 0.0689, 0.0688, 0.0501, 0.0489, 0.0703, 0.0587, 0.0728,
        0.0620, 0.1163, 0.0660, 0.0312, 0.0794, 0.0367, 0.0502, 0.0808, 0.0462,
        0.0717, 0.0988, 0.0404, 0.0331, 0.0386, 0.0839, 0.0901, 0.1307, 0.0602,
        0.0638, 0.0521, 0.0608, 0.0758, 0.0579, 0.0856, 0.0461, 0.0461, 0.0492,
        0.0678, 0.0356, 0.0408, 0.0397, 0.0450, 0.0652, 0.0491, 0.0573, 0.0675,
        0.0801, 0.0723, 0.1002, 0.0811, 0.0695, 0.0530, 0.0407, 0.0554, 0.0711,
        0.0775, 0.0697, 0.0725, 0.0509, 0.0923, 0.0948, 0.0468, 0.0381, 0.0322,
        0.0608, 0.0957, 0.0987, 0.0786, 